# Handling VERITAS spectra

After defining a data file format for VERITAS spectra -- https://github.com/chbrandt/veritas/blob/master/data_formatting-v2.rst -- we'll go here handle those files to see whether they fit our needs.

The files we are going to handle here are all from the same object: Mkn 421.
When an object is observed multiple times, under different activity levels, the VERITAS collaboration has chosen to split the corresponding *spectra* according to the object's activity.

Since an object can have multiple *spectra* associated with it, there must be a way to filter for the data points
of interest -- understanding the user will fine-tune the data according to its science case.
Straightforward solution for breaking down this degeneracy (same object for multiple tables) is to use the observations `epoch`; since no two events can occur at the same time and location, the moment of observation must be unique for each data file (i.e, *spectra*).
(To be precise, since the VHE astrophysics is a place where photon are somewhat rare, in a quiescent period it is necessary a long time to integrate the amount of light necessary to build up an *spectra*, if a *flare* happens during this period, the *low emission* *spectra* will contain the period of the flare; whereas the *high emission spectra* of the same object will correspond to the *flare* period only.)

When handlig the *spectra* datasets, the goal is to have a table with `energy`, `flux` and `epoch` so that plotting and modelling can be done.
Here, that is the place we have to arrive: (*i*) one plot `energy` *vs* `flux` and (*ii*) one plot `epoch` *vs* `flux`.

## Testing case: Mkn421

Seven data files with *spectra* from Mkn421 are available to build those SED plots.
The files are in ECSV format as proposed in https://github.com/chbrandt/veritas/blob/master/data_formatting-v2.rst.

In [1]:
%ls

handling_mkn421.ipynb          Mkn421_VERITAS_2008_low.csv
Mkn421_VERITAS_2008_highA.csv  Mkn421_VERITAS_2008_mid.csv
Mkn421_VERITAS_2008_highB.csv  Mkn421_VERITAS_2008_veryhigh.csv
Mkn421_VERITAS_2008_highC.csv  Mkn421_VERITAS_2008_verylow.csv


In [2]:
%cat 'Mkn421_VERITAS_2008_highA.csv'

# %ECSV 0.9
# ---
# meta: !!omap
# - object: Mrk 421
#
# - description:
#    Spectral points for multiwavelength campaign;
#    Observations taken between 2008 January 01 and 2008 June 05;
#    Flux sensitivity 0.8e-10 < flux(E>1TeV) < 1.1e-10
#
# - mjd:
#    start: 54502.46971
#    end: 54622.18955
#
# - article:
#    label: Ap.J. 738, 25 (2011)
#    url: http://iopscience.iop.org/0004-637X/738/1/25/
#    arxiv: http://arxiv.org/abs/1106.1210
#    ads: http://adsabs.harvard.edu/abs/2011ApJ...738...25A
#
# - comments:
#    - Name=Mrk421_2008_highA
#    - z=0.031
#    - LiveTime(h)=1.4
#    - significance=73.0
#
# - SED_TYPE: diff_flux_points
#
# datatype:
# - name: e_ref
#   unit: TeV
#   datatype: float64
# - name: dnde
#   unit: ph / (m2 TeV s)
#   datatype: float64
# - name: dnde_errn
#   unit: ph / (m2 TeV s)
#   datatype: float64
# - name: dnde_errp
#   unit: ph / (m2 TeV s)
#   datatype: float64
#
e_ref dnde       dnde_errn  dnde_errp
0.275 1.702E-005 3.295E-006 3.295E-006
0.340 

The format -- ECSV -- has been chosen for it is human readable and its metadata availability.
The library we have to use is Astropy.

In [3]:
from astropy.table import Table
tt = Table.read('Mkn421_VERITAS_2008_highA.csv', format='ascii.ecsv')

In [4]:
tt

e_ref,dnde,dnde_errn,dnde_errp
TeV,ph / (m2 s TeV),ph / (m2 s TeV),ph / (m2 s TeV)
float64,float64,float64,float64
0.275,1.702e-05,3.295e-06,3.295e-06
0.34,1.289e-05,1.106e-06,1.106e-06
0.42,8.821e-06,6.072e-07,6.072e-07
0.519,5.777e-06,3.697e-07,3.697e-07
0.642,3.509e-06,2.351e-07,2.351e-07
0.793,2.151e-06,1.525e-07,1.525e-07
0.98,1.302e-06,1.024e-07,1.024e-07
1.212,6.273e-07,6.117e-08,6.117e-08


In [5]:
import json
print json.dumps(tt.meta,indent=4)

{
    "object": "Mrk 421", 
    "description": "Spectral points for multiwavelength campaign; Observations taken between 2008 January 01 and 2008 June 05; Flux sensitivity 0.8e-10 < flux(E>1TeV) < 1.1e-10", 
    "mjd": {
        "start": 54502.46971, 
        "end": 54622.18955
    }, 
    "article": {
        "url": "http://iopscience.iop.org/0004-637X/738/1/25/", 
        "arxiv": "http://arxiv.org/abs/1106.1210", 
        "ads": "http://adsabs.harvard.edu/abs/2011ApJ...738...25A", 
        "label": "Ap.J. 738, 25 (2011)"
    }, 
    "comments": [
        "Name=Mrk421_2008_highA", 
        "z=0.031", 
        "LiveTime(h)=1.4", 
        "significance=73.0"
    ], 
    "SED_TYPE": "diff_flux_points"
}


### Push MJD to table

As noted before, the `epoch` is an important information for SED studies.
We may now transfer such information (here, `MJD`) to each data point in our table.

In [6]:
tt.meta['mjd']

{'end': 54622.18955, 'start': 54502.46971}

In [7]:
def mjd_header2table(table):
    def header2table(table, header_keyword, datatype=float, unitname='day'):
        from astropy.table import Column
        hfield = None
        colname = None
        if isinstance(header_keyword,(str,unicode)):
            hfield = table.meta[header_keyword]
            colname = header_keyword
        else:
            assert isinstance(header_keyword,(list,tuple))
            colname = '_'.join(header_keyword)
            for keyword in header_keyword:
                if hfield is None:
                    hfield = table.meta
                hfield = hfield.get(keyword)
        col = Column(data = [hfield]*len(table), 
                     name = colname, dtype=datatype, 
                     unit = unitname)
        table.add_column(col)

    header2table(table, ('mjd','start'))
    header2table(table, ('mjd','end'))
    table['mjd'] = table['mjd_start']
    table['mjd_delta'] = table['mjd_end'] - table['mjd_start']
    del table['mjd_start'],table['mjd_end']

mjd_header2table(tt)
tt

e_ref,dnde,dnde_errn,dnde_errp,mjd,mjd_delta
TeV,ph / (m2 s TeV),ph / (m2 s TeV),ph / (m2 s TeV),d,d
float64,float64,float64,float64,float64,float64
0.275,1.702e-05,3.295e-06,3.295e-06,54502.46971,119.71984
0.34,1.289e-05,1.106e-06,1.106e-06,54502.46971,119.71984
0.42,8.821e-06,6.072e-07,6.072e-07,54502.46971,119.71984
0.519,5.777e-06,3.697e-07,3.697e-07,54502.46971,119.71984
0.642,3.509e-06,2.351e-07,2.351e-07,54502.46971,119.71984
0.793,2.151e-06,1.525e-07,1.525e-07,54502.46971,119.71984
0.98,1.302e-06,1.024e-07,1.024e-07,54502.46971,119.71984
1.212,6.273e-07,6.117e-08,6.117e-08,54502.46971,119.71984


### Plotting the fluxes

Now that we have the `energy`, `flux` and `epoch` let's do those plots we want.

In [47]:
from bokeh.plotting import output_notebook
output_notebook()

from bokeh.plotting import show

class PlotFlux(object):
    flux = 'dnde'
    epoch = 'mjd'
    energy = 'e_ref'
    NUMAX = 16 # maximum number of datasets
    
    def __init__(self):
        super(PlotFlux,self).__init__()
        self.set_palette()
        self.glyphs = []
        
    def set_palette(self):
        from bokeh.palettes import viridis,magma
        colors = viridis(self.NUMAX/2)
        colors.extend(magma(self.NUMAX/2))
        from random import shuffle,seed
        seed(1234567)
        shuffle(colors)
        self.palette = colors[:]
        
    def set_units(self,table):
        self.flux_unit = table[self.flux].unit.to_string()
        self.epoch_unit = table[self.epoch].unit.to_string()
        self.energy_unit = table[self.energy].unit.to_string()

    def get_color(self,fig):
        from bokeh.models import GlyphRenderer
        num_sets = sum( isinstance(g,GlyphRenderer) for g in fig.renderers )/2
        colors = self.palette
        return colors[num_sets]

    def sed(self, table, fig = None):
        self.set_units(table)
        x_label = 'energy [{}]'.format(self.energy_unit)
        y_label = 'flux [{}]'.format(self.flux_unit)
        
        flux_errp = '{}_errp'.format(self.flux)
        flux_errn = '{}_errn'.format(self.flux)
        
        xs = table[self.energy]
        ys = table[self.flux]
        ys_errp = ys + table[flux_errp]
        ys_errn = ys - table[flux_errn]

        if fig is None:
            from bokeh.plotting import figure
            fig = figure(title='Spectral Energy Distribution',
                         x_axis_label=x_label,
                         y_axis_label=y_label,
                         y_axis_type = 'log',
                         height = 400,
                         width = 800)

        color = self.get_color(fig)
        
        # error first, to be placed below the main/measured data
        fig.multi_line(xs = zip(xs,xs),
                       ys = zip(ys_errn,ys_errp),
                       line_color=color)
        # central/measured data
        fig.circle(x = xs,
                   y = ys,
                   fill_color=color,
                      line_color=color)
        
        return fig
        
    def ted(self, table, fig = None):
        self.set_units(table)
        x_label = 'epoch [{}]'.format(self.epoch_unit)
        y_label = 'flux [{}]'.format(self.flux_unit)
        
        epoch_errp = '{}_delta'.format(self.epoch)

        from astropy.time import Time
        xs = Time(table[self.epoch],format='mjd').datetime
        xs_errp = Time(table[self.epoch] + table[epoch_errp], format='mjd').datetime
        ys = table[self.flux]
        
        if fig is None:
            from bokeh.models import DatetimeTickFormatter
            from bokeh.plotting import figure
            fig = figure(title='Time Emission Distribution',
                         x_axis_label=x_label,
                         y_axis_label=y_label,
                         y_axis_type = 'log',
                         x_axis_type = 'datetime',
                         height = 400,
                         width = 800)
            fig.xaxis.formatter=DatetimeTickFormatter(
                                                        hours=["%d %B %Y"],
                                                        days=["%d %B %Y"],
                                                        months=["%d %B %Y"],
                                                        years=["%d %B %Y"],
                                                    )
        
        color = self.get_color(fig)

        fig.multi_line(xs = zip(xs,xs_errp),
                       ys = zip(ys,ys),
                          line_color=color)

        fig.circle(x = xs,
                   y = ys,
                      fill_color=color,
                      line_color=color)
        
        return fig

Loading BokehJS ...

In [48]:
plot = PlotFlux()
_ = plot.sed(tt)
show(_)

In [49]:
_ = plot.ted(tt)
show(_)

## Adding another spectra

In [53]:
from glob import glob
table_files = glob('Mkn*.csv')

plot = PlotFlux()
fig = None
from astropy.table import Table
for fname in table_files:
    print "Reading file {}".format(fname)
    tt = Table.read(fname, format='ascii.ecsv')
    mjd_header2table(tt)
    fig = plot.sed(tt,fig)
show(fig)

Reading file Mkn421_VERITAS_2008_mid.csv
Reading file Mkn421_VERITAS_2008_low.csv
Reading file Mkn421_VERITAS_2008_highA.csv
Reading file Mkn421_VERITAS_2008_verylow.csv
Reading file Mkn421_VERITAS_2008_highB.csv
Reading file Mkn421_VERITAS_2008_highC.csv
Reading file Mkn421_VERITAS_2008_veryhigh.csv


In [54]:
from glob import glob
table_files = glob('Mkn*.csv')

plot = PlotFlux()
fig = None
from astropy.table import Table
for fname in table_files:
    print "Reading file {}".format(fname)
    tt = Table.read(fname, format='ascii.ecsv')
    mjd_header2table(tt)
    fig = plot.ted(tt,fig)
show(fig)

Reading file Mkn421_VERITAS_2008_mid.csv
Reading file Mkn421_VERITAS_2008_low.csv
Reading file Mkn421_VERITAS_2008_highA.csv
Reading file Mkn421_VERITAS_2008_verylow.csv
Reading file Mkn421_VERITAS_2008_highB.csv
Reading file Mkn421_VERITAS_2008_highC.csv
Reading file Mkn421_VERITAS_2008_veryhigh.csv
